<a href="https://colab.research.google.com/github/AlexAlexeyuk/Researchs/blob/master/CKD_mortalyty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 15 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.2
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_unknownUser_g1whcw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.148 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [196]:
data = h2o.import_file('/content/cleaned_data')
data['Surv'] = data['Surv'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [197]:
from h2o.automl import H2OAutoML


In [199]:
# Set predictors and response variable
response = "Surv"
predictors = data.col_names.remove(response)


In [200]:
# split
train, test = data.split_frame(seed = 1234, 
                                  ratios = [0.75], 
                                  destination_frames = ["train.hex", "test.hex"])

In [201]:
train.col_names

['Gender',
 'Age',
 'AF',
 'Localisation',
 'AMI_Unst.ang',
 'GFR',
 'Surv',
 'Q_non_Q',
 'DM']

In [202]:
aml = H2OAutoML(max_models = 6, max_runtime_secs = 120, seed = 12345)
aml.train(x = predictors, y = response, training_frame = train, leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [142]:
aml_ = H2OAutoML(max_models = 6, max_runtime_secs = 120, seed = 12345)
aml_.train(x = predictors, y = response, training_frame = train, leaderboard_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
aml.m

In [203]:
lb = aml.leaderboard

In [204]:
lb.head(rows=5)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GBM_1_AutoML_20201128_095618,0.909524,0.395232,0.896149,0.107143,0.326849,0.10683
DRF_1_AutoML_20201128_095618,0.895238,0.404048,0.861139,0.0714286,0.343073,0.117699
StackedEnsemble_BestOfFamily_AutoML_20201128_095618,0.885714,0.399114,0.83683,0.0714286,0.322372,0.103924
StackedEnsemble_AllModels_AutoML_20201128_095618,0.885714,0.410028,0.83683,0.0714286,0.343867,0.118245
GLM_1_AutoML_20201128_095618,0.866667,0.608981,0.8412,0.142857,0.411384,0.169236


In [146]:
aml.varimp_heatmap


<bound method varimp_heatmap of <h2o.automl.autoh2o.H2OAutoML object at 0x7fbb89f4fac8>>

In [206]:
import scipy
from scipy import stats
x = data[data['Surv'] == 'death']['GFR']
y = data[data['Surv'] == 'alive']['GFR']


In [207]:
y

GFR
98
69
86
52
47
21
46
21
75
66


In [209]:
x.name, y.name = 'death', 'alive'

In [214]:
x = [i for i in x]

In [ ]:
y = [i for i in y]
y

In [ ]:
res = stats.ttest_ind(x, y)
print('p-value: ', res[1])

In [221]:
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator

In [222]:
data.col_names

['Gender',
 'Age',
 'AF',
 'Localisation',
 'AMI_Unst.ang',
 'GFR',
 'Surv',
 'Q_non_Q',
 'DM']

In [227]:
stroke_coxph = H2OCoxProportionalHazardsEstimator(
                                                 stop_column="GFR",
                                                 ties="breslow")
stroke_coxph.train(x=predictors,
            y=response,
            training_frame=train)

coxph Model Build progress: |█████████████████████████████████████████████| 100%


In [228]:

pred = stroke_coxph.predict(test)

coxph prediction progress: |██████████████████████████████████████████████| 100%


In [230]:
stroke_coxph.coefficients_table

Model Details
H2OCoxProportionalHazardsEstimator :  Cox Proportional Hazards
Model Key:  CoxPH_model_python_1606551964032_5039

Call: 
Surv(GFR, Surv) ~ Gender + Age + AF + Localisation + AMI_Unst.ang + Q_non_Q + DM

Coefficients: CoxPH Coefficients


,names,coefficients,exp_coef,exp_neg_coef,se_coef,z_coef
0,Localisation.Right_car,0.392314,1.480402,0.675492,0.369326,1.062241
1,Localisation.Vertebro_bas,-0.100475,0.904408,1.105696,0.552015,-0.182014
2,AF.No_AF,-0.642116,0.526178,1.900497,0.364355,-1.762336
3,Gender.male,-0.477382,0.620405,1.611849,0.443702,-1.075907
4,AMI_Unst.ang.Unst_ang,-2.221644,0.108431,9.222478,0.794870,-2.794976
5,Q_non_Q.non_Q,-0.462309,0.629828,1.587735,0.412219,-1.121511
6,DM.No_Diabetes,-0.672368,0.510498,1.958871,0.372038,-1.807257
7,Age,0.005536,1.005551,0.994480,0.020572,0.269083



Likelihood ratio test=27.712016
n=84, number of events=46

Scoring History: 


,,timestamp,duration,iterations,loglik
0,,2020-11-28 10:06:25,0.000 sec,0,-178.081986
1,,2020-11-28 10:06:25,0.004 sec,1,-165.529541
2,,2020-11-28 10:06:25,0.007 sec,2,-164.291944
3,,2020-11-28 10:06:25,0.009 sec,3,-164.226643
4,,2020-11-28 10:06:25,0.013 sec,4,-164.225978


<bound method H2OCoxPHModel.coefficients_table of >